In [498]:
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import math
import re

In [546]:
data = pd.read_csv('test.csv')

In [547]:
data.pop('Unnamed: 0')
xx = data['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = data['Salary'].value_counts().sum()

In [548]:
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))

J'ai 726 Valeurs de salaires pour 3673 Na , avec un total de 4399


- Comment catégoriser les noms de Job, pour affiner les prédictions :

In [167]:
tmp = data['Title'].unique()
bla = []
bla2 = []
i=0
for i in tmp : 
    if 'data scientist' in i.lower() or 'datascientist' in i.lower() or 'scientist' in i.lower():
        bla.append('Data Scientist')
    elif 'machine learning' in i.lower() or 'artificial' in i.lower() or 'ia' in i.lower(): 
        bla.append('Machine Learning')
    elif 'data analyst' in i.lower() or 'analyste' in i.lower() or 'analyst' in i.lower(): 
        bla.append('Data Analyst')
    elif 'data engineer' in i.lower() or 'ingénieur data' in i.lower() or 'data ingénieur' in i.lower(): 
        bla.append('Data Engineer')
    elif 'big data' in i.lower() or 'cloud' in i.lower() : 
        bla.append('Big Data')
    elif 'manager' in i.lower() : 
        bla.append('Manager')
    elif 'security' in i.lower() or 'sécurité' in i.lower() : 
        bla.append('Cyber sécurity')
    elif 'consultant' in i.lower() : 
        bla.append('Consultant')
    elif 'data' in i.lower() or 'bi' in i.lower() or 'business inteligence' in i.lower() or 'business developper' in i.lower() : 
        bla.append('Data')
    elif 'ingenieur' in i.lower() or 'ingénieur' in i.lower() or 'engineer' in i.lower(): 
        bla.append('Ingénieur')
    elif 'web' in i.lower() or 'fullstack'in i.lower().replace('-','').replace(' ','') or 'intégrateur' in i.lower() : 
        bla.append('Dev Web')
    elif 'web' in i.lower() or 'frontend'in i.lower().replace('-','').replace(' ','') or 'javascript'in i.lower() : 
        bla.append('Dev Front end')
    elif 'php'in i.lower() or  'backend'in i.lower().replace('-','').replace(' ','')  : 
        bla.append('Dev Back end')
    elif 'développeur' in i.lower() or 'developpeur' in i.lower() or 'développeu' in i.lower() or 'developer' in i.lower(): 
        bla.append('developpeur')
    else : 
        bla.append(i)
d = pd.Series(bla).value_counts()
d

developpeur                                           231
Data Analyst                                          176
Dev Web                                               117
Ingénieur                                             111
Data Scientist                                        101
                                                     ... 
Conseiller.ère Clientèle - Agence de Monplaisir         1
Chargé d’Affaires Ingénierie H/F par Humanys            1
Architecte d'entreprise H/F                             1
Responsable Qualité à temps partiel F/H                 1
Stage - Bras droit CEO - Freelance-Stack.io - Lyon      1
Length: 199, dtype: int64

- Drop les doublons : 

In [582]:
def getdata(urlfiles):
    data = pd.read_csv(urlfiles)
    data.pop('Unnamed: 0')
    data = data.drop_duplicates(subset=['Link'], keep='first')
    return data
data = getdata('test.csv')
xx = data['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = data['Salary'].value_counts().sum()
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))

J'ai 554 Valeurs de salaires pour 2290 Na , avec un total de 2844


- Comment Catégoriser les dates : 

In [583]:
def MakeDateGreatAgain(df):
    dede = []
    for k,i in df.iterrows() : 
        if i['Date'].lower().replace(' ','') == "publiéeàl'instant" or i['Date'].lower().replace(' ','') == "aujourd'hui":
            d = datetime.today()
            df['Date'][k] = d
        elif i['Date'].lower().replace(' ','') == "ilya30+jours":
            d = datetime.today() - timedelta(days=31)
            df['Date'][k] = d
        else: 
            if len(i['Date'].split()) == 5 :
                if i['Date'].split()[3].isdigit():
                    d = datetime.today() - timedelta(days=int(i['Date'].split()[3]))
                    df['Date'][k] = d
                else : 
                    df.drop(k)
            else :
                dede.append(k)
    return df.drop(dede)

data = MakeDateGreatAgain(data)

- Definir les types de contrats

In [584]:
def Makedummiestypecontrat(data):
    doto = pd.DataFrame(columns= ['TypeContrat_CDI','TypeContrat_Temps plein','TypeContrat_Apprentissage','TypeContrat_Contrat pro','TypeContrat_CDD','TypeContrat_Intérim','TypeContrat_Temps partiel','TypeContrat_Stage','TypeContrat_Indépendant','TypeContrat_notfound'])
    for key ,row in data.iterrows():
        for typecont in row['TypeContrat'].split(','):
            if typecont.strip() == 'Freelance / Indépendant':
                namecol = 'TypeContrat_'+ 'Indépendant'
                doto.loc[key]= 0
                doto[namecol][key] =1
            else :
                namecol = 'TypeContrat_'+ typecont.strip()
                doto.loc[key]= 0
                doto[namecol][key] =1
                
    data = pd.concat([data, doto],axis=1)
    data = data.drop('TypeContrat',axis=1)
    return data
       
data = Makedummiestypecontrat(data)

- définir la moyenne des salaires

In [585]:
def getmeansalary(df):
    for key ,row in df.iterrows():
        if row['Salary'] == "notfound":
            df['Salary'][key] = 'notfound'
        else: 
            non_decimal = re.compile(r'[^\d.]+')
            separatenum = row['Salary'].split('-')
            if len(separatenum) == 1 :
                if 'par an' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)
                elif 'par mois' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*12
                elif 'par jour' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*22*12
                elif 'par semaine' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*4*12
                elif 'par heure' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*8*22*12
            else: 
                if 'par an' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche),int(hauttranche)])
                    df['Salary'][key] = moyennealannee
                elif 'par mois' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*12,int(hauttranche)*12])
                    df['Salary'][key] = moyennealannee
                elif 'par jour' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*22*12,int(hauttranche)*22*12])
                    df['Salary'][key] = moyennealannee
                elif 'par semaine' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*4*12,int(hauttranche)*4*12])
                    df['Salary'][key] = moyennealannee
                elif 'par heure' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*8*22*12,int(hauttranche)*8*22*12])
                    df['Salary'][key] = moyennealannee
                else: 
                    print("Error : Tout n'est pas passé ")
    return df
                

data = getmeansalary(data)

C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [580]:
data

,Title,Company,Location,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant,TypeContrat_notfound
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris 9e (75),notfound,Contrat\nALTERNANCE\nFinalité du poste\nAu sei...,2020-05-15 15:59:57.038461,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,0,1,0,0,0,0,0,0
1,FULL REMOTE Senior Data Scientist H/F - CDI,Jellysmack,Levallois-Perret (92),notfound,Nous continuons de recruter et avons adapté no...,2020-04-24 15:59:57.039456,/pagead/clk?mo=r&ad=-6NYlbfkN0B8n3TtewkfrSQLVL...,1,0,0,0,0,0,0,0,0,0
2,Chef de projets ERP,Castelis,94200 Ivry-sur-Seine,50000,"Présentation de la société Castelis conçoit, d...",2020-04-19 15:59:57.039456,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0,1
3,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F,BNP Paribas,Levallois-Perret (92),notfound,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F (N...,2020-05-19 15:59:57.040445,/rc/clk?jk=dc84057ced14dfad&fccid=1392500f835b...,0,0,0,0,0,0,0,1,0,0
4,Data Scientist H/F,NEOBRAIN,75009 Paris 9e,notfound,Neobrain développe des solutions digitales int...,2020-05-19 15:59:57.040445,/company/NEOBRAIN/jobs/Data-Scientist-dfce81a4...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,Développeur Photovoltaïque - NANTES F/H,AMARENCO CONSTRUCTION,Nantes (44),notfound,Dans le cadre d’un renforcement de l’activité ...,2020-04-18 15:59:57.679090,/rc/clk?jk=4332dda4f8e69264&fccid=b1db990939bf...,1,0,0,0,0,0,0,0,0,0
4395,Développeur/Configurateur Salesforce.com F/H -...,Accenture,Nantes (44),notfound,Rejoignez le centre de service d’Accenture Tec...,2020-04-18 15:59:57.679090,/rc/clk?jk=db587b3b19155869&fccid=a4e4e2eaf266...,0,0,0,0,0,0,0,0,0,1
4396,Développeur intranet,Le Groupe La Poste,Nantes (44),notfound,MISSION\nLa DSI de La Banque Postale et du Rés...,2020-04-18 15:59:57.680090,/rc/clk?jk=a9e7f8871cedb756&fccid=603bc4f64b36...,0,0,0,0,0,0,0,0,0,1
4397,Développeur foncier (H/F),Vilogia Premium,Nantes (44),notfound,"Descriptif du poste\nVilogia Premium, filiale ...",2020-04-18 15:59:57.680090,/rc/clk?jk=60f7e3fbd8c241a7&fccid=6e05e77f4f65...,1,0,0,0,0,0,0,0,0,0


In [581]:
def getpreprocessdata(urlfiles):
    df = getdata(urlfiles)
    df = MakeDateGreatAgain(df)
    df = Makedummiestypecontrat(df)
    df= getmeansalary(df)
    return df

data = getpreprocessdata('test.csv')
data

C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,Title,Company,Location,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant,TypeContrat_notfound
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris 9e (75),notfound,Contrat\nALTERNANCE\nFinalité du poste\nAu sei...,2020-05-15 16:01:38.236809,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,0,1,0,0,0,0,0,0
1,FULL REMOTE Senior Data Scientist H/F - CDI,Jellysmack,Levallois-Perret (92),notfound,Nous continuons de recruter et avons adapté no...,2020-04-24 16:01:38.237809,/pagead/clk?mo=r&ad=-6NYlbfkN0B8n3TtewkfrSQLVL...,1,0,0,0,0,0,0,0,0,0
2,Chef de projets ERP,Castelis,94200 Ivry-sur-Seine,50000,"Présentation de la société Castelis conçoit, d...",2020-04-19 16:01:38.237809,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0,1
3,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F,BNP Paribas,Levallois-Perret (92),notfound,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F (N...,2020-05-19 16:01:38.237809,/rc/clk?jk=dc84057ced14dfad&fccid=1392500f835b...,0,0,0,0,0,0,0,1,0,0
4,Data Scientist H/F,NEOBRAIN,75009 Paris 9e,notfound,Neobrain développe des solutions digitales int...,2020-05-19 16:01:38.237809,/company/NEOBRAIN/jobs/Data-Scientist-dfce81a4...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,Développeur Photovoltaïque - NANTES F/H,AMARENCO CONSTRUCTION,Nantes (44),notfound,Dans le cadre d’un renforcement de l’activité ...,2020-04-18 16:01:38.847863,/rc/clk?jk=4332dda4f8e69264&fccid=b1db990939bf...,1,0,0,0,0,0,0,0,0,0
4395,Développeur/Configurateur Salesforce.com F/H -...,Accenture,Nantes (44),notfound,Rejoignez le centre de service d’Accenture Tec...,2020-04-18 16:01:38.847863,/rc/clk?jk=db587b3b19155869&fccid=a4e4e2eaf266...,0,0,0,0,0,0,0,0,0,1
4396,Développeur intranet,Le Groupe La Poste,Nantes (44),notfound,MISSION\nLa DSI de La Banque Postale et du Rés...,2020-04-18 16:01:38.847863,/rc/clk?jk=a9e7f8871cedb756&fccid=603bc4f64b36...,0,0,0,0,0,0,0,0,0,1
4397,Développeur foncier (H/F),Vilogia Premium,Nantes (44),notfound,"Descriptif du poste\nVilogia Premium, filiale ...",2020-04-18 16:01:38.847863,/rc/clk?jk=60f7e3fbd8c241a7&fccid=6e05e77f4f65...,1,0,0,0,0,0,0,0,0,0


In [597]:
i=0
j= 0
k=0
for key, row in data.iterrows() : 
    if 'junior' in row['Description'].lower():
        i+=1
    elif 'senior' in row['Description'].lower():
        k+=1
    elif 'junior' in row['Title'].lower():
        i+=1
    elif 'senior' in row['Title'].lower():
        k+=1
    else : 
        for phrase in row['Description'].strip('.') :
            if 'Experiences:' in phrase.strip():
                
        j+=1
print(i, 'junior allowed -', k, 'Senior needed -', j , 'notfound')

144 junior allowed - 281 Senior needed - 2390 notfound


In [599]:
pd.read_csv('data/pierre.csv',sep=',')

,_id,query,title,company,rating_mean,location,salary,summary,sponso,date,rating_count,contract,desc
0,f511c22c063327a2,"[""data+scientist""]",Data Scientist H/F - Alternance - Niort,MAIF,"3,5",Niort (79),NaN,Faire tourner le modèle sur un jeu de conversa...,Sponsorisée,Il y a plus de 30 jours,86 avis,"Apprentissage, Contrat pro",Intitulé du poste\nData Scientist H/F - Altern...
1,ee8a41c8db4c0bcf,"[""data+scientist""]",Data Scientist Marketing H/F,Bouygues Telecom,"3,8",Meudon (92),NaN,"Au sein de l'équipe Connaissance Client, compo...",Sponsorisée,Il y a plus de 30 jours,640 avis,"Temps plein, CDI","Au sein de l'équipe Connaissance Client, compo..."
2,a8284ae0fe30bcb8,"[""data+scientist""]",Développeur JAVA / Data Scientist,Amiltone,"5,0",Grenoble (38),NaN,Le poste est basé à GRENOBLE (38).\nLe dévelop...,Sponsorisée,Publiée à l'instant,2 avis,NaN,Localisation :\nLe poste est basé à GRENOBLE (...
3,742278d478124f60,"[""data+scientist""]",CDI - Data scientist senior H/F,BIOMERIEUX sa,"3,8",Craponne (69),NaN,Acteur mondial dans le domaine du diagnostic i...,Sponsorisée,il y a 19 jours,440 avis,NaN,Acteur mondial dans le domaine du diagnostic i...
4,cb00bc9002770a71,"[""data+scientist""]",Data Scientist h/f- Paris- Cdi,Robert Half France,"3,9",Paris (75),NaN,Ils sont à la recherche de passionné(e)s de la...,Sponsorisée,Il y a plus de 30 jours,"12,114 avis",CDI,A propos de notre client\nNotre client est une...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,8fec4412a0f9e94d,"[""data+scientist""]",DATA ENGINEER H/F,Groupe AFG,NaN,Lille (59),NaN,"Modéliser, mettre en place et industrialiser d...",NaN,Il y a plus de 30 jours,NaN,NaN,Le poste\nDans le cadre du développement de no...
380,4e49eae355fb39b6,"[""data+scientist""]",Senior Data Scientist (Paris and Berlin),PriceHubble,NaN,Paris (75),NaN,"As a data scientist, your mission will be to d...",NaN,Il y a plus de 30 jours,NaN,NaN,"PriceHubble is a PropTech company, set to radi..."
381,5fe0dbf8caffa80a,"[""data+scientist""]",Chef de projet fonctionnel (IT) H/F,CACIIS,NaN,Paris 15e (75),36 000 € - 50 000 € par an,"Des bases de données (SQL, Big data), Script/D...",NaN,Il y a plus de 30 jours,NaN,"Temps plein, CDI","ENTREPRISE\nDepuis 2007, CACIIS est un véritab..."
382,6c919c10a85baaeb,"[""data+scientist""]",Financial & Planning Analyst,Deezer,"3,9",Paris (75),NaN,"From data scientists to tech experts, artists ...",NaN,Il y a plus de 30 jours,8 avis,NaN,Company Description\nWe are music and tech fan...
